# Неявное решение уравнения фильтрации однофазной жидкости в двухмерном пласте

Муравцев Александр. Вариант 16

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Задание значений входных параметров и перевод единиц измерения в СИ

In [2]:
mesh_x_count = 4  # количество ячеек по пространству в направлении оси Ox
mesh_y_count = 4  # количество ячеек по пространству в направлении оси Oy
mesh_t_count = 1000  # количество шагов по времени

In [3]:
dt = 10  # сут, временной шаг
dt *= 24 * 60 * 60  # с, временной шаг

m = 0.18  # пористость

mu = 4  # мПа*с, вязкость жидкости
mu *= 1e-3  # Па*с, вязкость жидкости

k = np.array([
    [30.0, 30.0, 30.0, 30.0],
    [30.0, 40.0, 40.0, 40.0],
    [30.0, 40.0, 40.0, 50.0],
    [30.0, 40.0, 50.0, 40.0]
])  # мД, проницаемость
k *= 1e-3 * (1e-6) ** 2  # м^2, проницаемость

dx = 100  # м, ширина ячейки по оси Ox
dy = 100  # м, ширина ячейки по оси Oy

h = 50  # м, толщина пласта

b_liq = 1  # м^3/м^3, объёмный коэффициент жидкости

compr = 2.2e-9  # Па^(-1), общая сжимаемость

p_initial = 20  # МПа, начальное пластовое давление
p_initial *= 1e6  # Па, начальное пластовое давление

q_inj = 40  # м^3/сут, расход нагнетательной скважины в поверхностных условиях
q_inj /= (24 * 60 * 60)  # м^3/с, расход нагнетательной скважины в поверхностных условиях

q_prod = 40  # м^3/сут, дебит добывающей скважины в поверхностных условиях
q_prod /= (24 * 60 * 60)  # м^3/с, дебит добывающей скважины в поверхностных условиях

In [4]:
# безразмерные множители, входящие в формулы неяного метода
gamma = (m * mu * compr / k) * (dx ** 2 / dt)
beta = (dx / dy) ** 2

In [5]:
p = []
p.append(np.full((mesh_x_count, mesh_y_count), p_initial))

In [6]:
system_matrix = np.zeros((mesh_x_count * mesh_y_count, mesh_x_count * mesh_y_count))
d = np.zeros(mesh_x_count * mesh_y_count)

for i in range(mesh_x_count):
    for j in range(mesh_x_count):
        system_matrix[i * mesh_x_count + j, i * mesh_x_count + j] = -2 - 2 * beta - gamma[i, j]
        d[i * mesh_x_count + j] = -gamma[i, j] * p[-1][i, j] 
system_matrix += np.diag([1] * (mesh_x_count * mesh_y_count - 1), k=-1)
system_matrix += np.diag([1] * (mesh_x_count * mesh_y_count - 1), k=1)
system_matrix += np.diag([beta] * (mesh_y_count * mesh_x_count - mesh_x_count), k=mesh_x_count)
system_matrix += np.diag([beta] * (mesh_y_count * mesh_x_count - mesh_x_count), k=-mesh_x_count)
system_matrix

array([[-4.61111111,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        , -4.61111111,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  1.        , -4.61111111,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        , -4.61111111,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.        ,  0

In [7]:
np.linalg.solve(system_matrix, d).reshape(mesh_x_count, mesh_y_count)

array([[ 7107323.2523102 ,  8857879.01307603,  9371313.11789145,
         9877081.39582625],
       [11692555.98368765, 12143805.74564892, 12254983.41248607,
        12258228.44584195],
       [12565242.19766326, 12169048.87343504, 11696787.07128471,
        10787294.81506979],
       [ 9597672.38626039,  8681174.54613433,  7770181.92515345,
         6218499.455751  ]])

In [8]:
p[-1]

array([[20000000., 20000000., 20000000., 20000000.],
       [20000000., 20000000., 20000000., 20000000.],
       [20000000., 20000000., 20000000., 20000000.],
       [20000000., 20000000., 20000000., 20000000.]])